# Compute resonance energies and strength in the DR if lithium-like ions

In [1]:
using JAC

┌ Info: Recompiling stale cache file /home/fritzsch/.julia/compiled/v1.0/JAC/vyEUL.ji for JAC [830ae420-d14d-11e8-2f94-6b071437414d]
└ @ Base loading.jl:1190



Welcome to JAC:  A community approach to the computation of atomic structures, cascades and time evolutions [(C) Copyright by Stephan Fritzsche, Jena (2018-2020)].



The dielectronic recombination of ions is known as as *resonant capture process*, in which an electron is first captured resonantly from the continuum into a doubly-excited (N+1 electron) resonance level and subsequently stabilizes by the emission of photons to some (final N+1 electron) level. Since the excited resonance is embedded into the continuum of the initial ion and with often two or more excited electrons, the first part of a dielectronic recombination is the *inverse* process to an autoionization. The strength of the resonances is depends on the radiative rate, relative to the total decay rate, i.e. the sum of the radiative and re-autoionization rates. For light and medium ions, the excited ion often returns by autoionization back into the inital charge stage, and before a radiative stabilization would occur.

Most often, the dielectronic recombination is described as a two-step (cascade) process in which Auger and radiative (emission) amplitudes and rates need to be combined. To compute these amplitudes, one needs to distinguish between the initial levels (of the N-electron ion) and the intermediate (resonance) and final-state levels (of the N+1 electron ion). The relative level energies of these three multiplets decide, whehther and with which individual strength resonances occur in the spectra, and which correspond to some given electron configurations of the intermediate levels. Often, the pronounced resonances refer to quite high excitations of (at least) one of the electrons.

Here, we shall compute the resonance energies and strength for the dielectronic capture of initially lithium-like ions and, especially, lithlium-like argon $1s^2 2s\quad \to\quad [1s 2s^2 2p + 1s 2s 2p^2]\quad \to\quad 1s^2 2s^2 + 1s^2 2s 2p \quad$. As before, we wish to perform an `Atomic.Computation`, for which we specify the given initial-, intermediate- and final-state configurations. 

Before we do so, let us start from the `Dielectronic.Settings` in order to understand how we control such computations:

In [2]:
? Dielectronic.Settings

`struct  Dielectronic.Settings`       ... defines a type for the details and parameters of computing dielectronic recombination pathways.

```
+ multipoles            ::Array{EmMultipoles}  ... Multipoles of the radiation field that are to be included.
+ gauges                ::Array{UseGauge}      ... Specifies the gauges to be included into the computations.
+ printBefore           ::Bool                 ... True, if all energies and pathways are printed before their eval.
+ selectPathways        ::Bool                 ... True if particular pathways are selected for the computations.
+ selectedPathways      ::Array{Tuple{Int64,Int64,Int64},1}  
                                               ... List of pathways, given by tupels (inital, inmediate, final).
+ electronEnergyShift   ::Float64              ... An overall energy shift for all electron energies (i.e. from the 
                                                   initial to the resonance levels.
+ photonEnergyShift     ::Float64              ... An overall energy shift for all photon energies (i.e. from the 
                                                   resonance to the final levels.
+ mimimumPhotonEnergy   ::Float64              ... minimum transition energy for which photon transitions are 
                                                   included into the evaluation.
+ augerOperator         ::String               ... Auger operator that is to be used for evaluating the Auger ampl's: 
                                                   allowed values are: "Coulomb", "Breit", "Coulomb+Breit"
```

---

`Dielectronic.Settings()`       ... constructor for the default values of dielectronic recombination pathway computations.

---

`(set::Dielectronic.Settings;`

```
    multipoles=..,          gauges=..,                  printBefore=..,             selectPathways=..,             
    selectedPathways=..,    electronEnergyShift=..,     photonEnergyShift=..,       mimimumPhotonEnergy=..,
    augerOperator=..)
                
... constructor for modifying the given Dielectronic.Settings by 'overwriting' the previously selected parameters.
```


In [3]:
Dielectronic.Settings()

multipoles:                 EmMultipole[E1]  
use-gauges:                 UseGauge[]  
printBefore:                false  
selectPathways:             false  
selectedPathways:           Tuple{Int64,Int64,Int64}[]  
electronEnergyShift:        0.0  
photonEnergyShift:          0.0  
mimimumPhotonEnergy:        0.0  
augerOperator:              Coulomb  


From these `Settings`, we see that we can (need to) control both, the `augerOperator` for the capture process as well as the `multipoles` and `gauges` for the radiative stabilization. The meaning of these parameters is completely analogue to the related Auger and photon emission processes in the previous tutorials. In contrast to the (atomic) lines of these processes, however, we now have to deal with (two-step) *DR pathways*, i.e. level triples (initial-intermediate-final). These triples can be specified as before by choosing `selectPathways = true` and by providing a corresponding list `selectedPathways` of such triples. This feature is often helpful to restrict the computations to a few selected pathways. In addition, all the electron and photon energies can be *shifted* can be shifted by a constant amount, a feature that may help to assign the resonances properly. For our example above, we shall take the following settings:

In [4]:
drSettings = Dielectronic.Settings([E1, M1], [JAC.UseCoulomb, JAC.UseBabushkin], true, 
                                   false, Tuple{Int64,Int64,Int64}[], 0., 0., 0., "Coulomb")

multipoles:                 EmMultipole[E1, M1]  
use-gauges:                 UseGauge[UseCoulomb, UseBabushkin]  
printBefore:                true  
selectPathways:             false  
selectedPathways:           Tuple{Int64,Int64,Int64}[]  
electronEnergyShift:        0.0  
photonEnergyShift:          0.0  
mimimumPhotonEnergy:        0.0  
augerOperator:              Coulomb  


Apart from the settings, some special care has to be taken for the chosen grid as we need to represent both, the doubly-excited resonances as well as the continuum orbitals, properly. We therefore choose again a logarithmic-linear grid and define the atomic computation as:

In [5]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-5,h = 5.0e-2, hp = 1.0e-2, NoPoints = 900)

Radial grid:  rnt = 2.0e-5,  h = 0.05,  hp = 0.01,  NoPoints = 900, orderL = 7,  orderS = 8,  nsL = 134,  nsS = 135,  mesh = JAC.Radial.MeshGL(), ...  
r:    [0.0, 2.13242e-7, 1.08301e-6]  ...  [6.47176, 6.48316, 6.4902]
wr:   [0.0, 5.42552e-7, 1.17199e-6]  ...  [0.0129626, 0.00949561, 0.00439584]
t:    [0.0, 0.0, 0.0]  ...  [6.49193, 6.49193, 6.49193]

In [6]:
wa = Atomic.Computation(Atomic.Computation(), name="DR lithium-like argon", grid=grid, nuclearModel=Nuclear.Model(18.), 
                        initialConfigs     =[Configuration("1s^2 2s")],
                        intermediateConfigs=[Configuration("1s 2s^2 2p"), Configuration("1s 2s 2p^2") ],  
                        finalConfigs       =[Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p")],  
                        process = Dierec(), processSettings = drSettings  )
wb = perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^1 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704537e+02    -1.62704858e+02    +1.97540696e-06    
      2      2s_1/2      -4.07203231e+01    -4.07203638e+01    +1.00176044e-06    
      3      3s_1/2      -1.80782718e+01    -1.80782839e+01    +6.69151399e-07    
      4      4s_1/2      -1.01607481e+01    -1.01607532e+01    +

While the dielectronic recombination of lithium-like ions via the 1s^-1 hole-state resonances is always possible, if free electron with sufficient high energies (~2.2 keV in the example above) are available, such DR processes may occur also at lower energies via excitations into Rydberg orbitals. For the $2s\;\to\; 2p$ excitation of lihtium-like tungsten (Z=74), a dielectronic recombination is possible if electrons are captured into the $7l$ shells for $2s\;\to\; 2p_1/2$ excitations and into the $19l$ shells for $2s\;\to\; 2p_3/2$ excitations, respectively. The modelling of such DR spectra places higher demands on the computations since a proper set of configurations need to be chosen for both, the intermediate- and final-state levels. Here, we do not attempt to perform a detailed computation of this DR spectrum but just indicate how they can still be -- quite readily -- be computed by means of JAC.


In the following, we consider lithlium-like tungsten $1s^2 2s\quad \to\quad [1s^2 2p 7s + 1s^2 2p 7p + 1s^2 2p 7d + 1s^2 2p 7s + 1s^2 2p 7f]\quad \to\quad 1s^2 2s^2 + 1s^2 2s 2p + 1s^2 2s 7s + 1s^2 2s 7p + 1s^2 2s 7d + 1s^2 2s 7f  \quad$. Although the stabilization of the ions via $ 2p\:\to\: 2s$ transitions might be dominant, this simplified computations omits many decay channels due to transitions of the $7l$ electron into the $3l,\; 4l$, etc. levels. For lithlium-like tungsten, moreover, one might expect rather strong relativistic contributions to the binding (transition) energies and the transition amplitudes. Here, we simply omit all these contributions and just compute:

In [7]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-6,h = 2.0e-2, hp = 5.0e-2, NoPoints = 2000)

Radial grid:  rnt = 2.0e-6,  h = 0.02,  hp = 0.05,  NoPoints = 2000, orderL = 7,  orderS = 8,  nsL = 292,  nsS = 293,  mesh = JAC.Radial.MeshGL(), ...  
r:    [0.0, 7.64774e-9, 3.88411e-8]  ...  [57.4144, 57.4707, 57.5055]
wr:   [0.0, 1.94582e-8, 4.20324e-8]  ...  [0.0640289, 0.0469037, 0.0217133]
t:    [0.0, 0.0, 0.0]  ...  [57.514, 57.514, 57.514]

In [8]:
wc = Atomic.Computation(Atomic.Computation(), name="DR lithium-like tungsten: 2s -> 2p_1/2", grid=grid, nuclearModel=Nuclear.Model(74.0), 
                        initialConfigs     = [Configuration("1s^2 2s"), Configuration("1s^2 2p"), Configuration("1s^2 7s"), 
                                              Configuration("1s^2 7p"), Configuration("1s^2 7d"), Configuration("1s^2 7f")],
                        intermediateConfigs= [Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p"), Configuration("1s^2 2p 7s"), 
                                              Configuration("1s^2 2p 7p"), Configuration("1s^2 2p 7d"), Configuration("1s^2 2p 7f")],  
                        finalConfigs       = [Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p"), Configuration("1s^2 2s 7s"), 
                                              Configuration("1s^2 2s 7p"), Configuration("1s^2 2s 7d"), Configuration("1s^2 2s 7f")],
                        process = Dierec(), processSettings=drSettings )

wd = perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^1 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704537e+02    -1.62704858e+02    +1.97540696e-06    
      2      2s_1/2      -4.07203231e+01    -4.07203638e+01    +1.00176044e-06    
      3      3s_1/2      -1.80782718e+01    -1.80782839e+01    +6.69151399e-07    
      4      4s_1/2      -1.01607481e+01    -1.01607532e+01    +